<a href="https://colab.research.google.com/github/Hu-Hao/quant-learning/blob/main/examples/colab_example_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quantitative Trading Framework - Complete Example

This notebook demonstrates:
- **Our Custom Framework**: Complete backtesting with real market data
- **VectorBT Cross-Validation**: Compare results with industry-standard library
- **Side-by-Side Analysis**: Verify strategy implementation accuracy

## 1. Setup and Installation

In [ ]:
# Install required packages
!pip install yfinance pandas numpy matplotlib seaborn vectorbt

# Clone the quantitative trading framework from GitHub
!git clone https://github.com/Hu-Hao/quant-learning.git

# Add the project to Python path
import sys
sys.path.append('/content/quant-learning')

print("✅ Installation completed successfully!")

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Import our framework
from quant_trading.strategies.moving_average import MovingAverageStrategy
from quant_trading.backtesting.engine import BacktestEngine
from quant_trading.utils.vectorbt_comparison import (
    compare_with_vectorbt, 
    create_vectorbt_comparison_plots
)

# Set up plotting
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ All libraries imported successfully!")

## 2. Data Preparation

In [ ]:
# Fetch Apple stock data
symbol = "AAPL"
end_date = datetime.now()
start_date = end_date - timedelta(days=365)  # 1 year of data

print(f"Fetching {symbol} data from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}...")

ticker = yf.Ticker(symbol)
data = ticker.history(start=start_date, end=end_date)
data.columns = [col.lower() for col in data.columns]  # Lowercase for compatibility

print(f"✅ Fetched {len(data)} days of data")
print(f"Price range: ${data['low'].min():.2f} - ${data['high'].max():.2f}")
print(f"Total return: {(data['close'].iloc[-1]/data['close'].iloc[0] - 1)*100:.1f}%")

In [ ]:
# Visualize the data
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Price chart
ax1.plot(data.index, data['close'], label='Close Price', linewidth=1.5, color='blue')
ax1.plot(data.index, data['close'].rolling(20).mean(), label='20-day MA', linewidth=1, color='orange')
ax1.plot(data.index, data['close'].rolling(50).mean(), label='50-day MA', linewidth=1, color='red')
ax1.set_title(f'{symbol} Stock Price with Moving Averages')
ax1.set_ylabel('Price ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Volume chart
ax2.bar(data.index, data['volume'], alpha=0.7, color='green')
ax2.set_title(f'{symbol} Trading Volume')
ax2.set_ylabel('Volume')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Strategy Setup

In [ ]:
# Create Moving Average strategy
strategy = MovingAverageStrategy(
    short_window=10,
    long_window=30,
    quantity=100
)

# Backtest parameters
initial_capital = 100000  # $100,000
commission = 0.001        # 0.1%
slippage = 0.001         # 0.1%

print(f"📊 Strategy: {strategy.get_name()}")
print(f"Parameters: {strategy.get_parameters()}")
print(f"Initial capital: ${initial_capital:,}")
print(f"Commission: {commission:.1%}")

## 4. Our Framework Backtest

In [ ]:
# Run backtest with our framework
print("🚀 Running backtest with our framework...")

engine = BacktestEngine(
    initial_capital=initial_capital,
    commission=commission,
    slippage=slippage,
    allow_short_selling=False  # Beginner-friendly
)

engine.run_backtest(data, strategy)
performance = engine.get_performance_summary()

# Display results
final_value = engine.portfolio_values[-1]
total_return = (final_value / initial_capital - 1) * 100
buy_hold_return = (data['close'].iloc[-1] / data['close'].iloc[0] - 1) * 100

print(f"\n📊 OUR FRAMEWORK RESULTS:")
print(f"   Final Value: ${final_value:,.2f}")
print(f"   Total Return: {total_return:.2f}%")
print(f"   Buy & Hold: {buy_hold_return:.2f}%")
print(f"   Outperformance: {total_return - buy_hold_return:+.2f}%")
print(f"   Total Trades: {len(engine.trades)}")
print(f"   Sharpe Ratio: {performance.get('sharpe_ratio', 0):.3f}")
print(f"   Max Drawdown: {performance.get('max_drawdown', 0)*100:.1f}%")

In [ ]:
# Create comprehensive visualization
fig = engine.plot_backtest_results(
    strategy_name="Moving Average Strategy",
    symbol=symbol,
    show_plot=True
)

if fig:
    print("✅ Comprehensive backtest visualization created!")
    print("\n🔍 Chart Interpretation:")
    print("   • Green ▲ = Buy signals")
    print("   • Red ▼ = Sell signals")
    print("   • Blue line = Strategy performance")
    print("   • Gray line = Buy & Hold benchmark")
else:
    print("⚠️ Visualization not available")

## 5. VectorBT Cross-Validation

Now let's validate our results by comparing with VectorBT, a professional backtesting library.

In [ ]:
# Run cross-validation comparison
print("🔄 VECTORBT CROSS-VALIDATION")
print("=" * 40)

print(f"Comparing our framework vs VectorBT...")
print(f"Same strategy, same data, same parameters")

# Run comprehensive comparison
comparison_results = compare_with_vectorbt(
    data=data,
    strategy=strategy,
    initial_capital=initial_capital,
    commission=commission,
    show_details=True
)

# Extract results
our_results = comparison_results['our_results']
vbt_results = comparison_results['vectorbt_results']
comparison = comparison_results['comparison']

if our_results['success'] and vbt_results['success']:
    print(f"\n✅ VALIDATION SUCCESSFUL!")
    print(f"\n📊 Performance Comparison:")
    print(f"   Our Framework: {our_results['performance']['total_return']*100:.2f}%")
    print(f"   VectorBT:      {vbt_results['performance']['total_return']*100:.2f}%")
    print(f"   Difference:    {comparison['return_difference']*100:.2f}%")
    print(f"   Correlation:   {comparison['correlation']:.3f}")
    print(f"   Status:        {comparison['status']}")
    
    if comparison['insights']:
        print(f"\n💡 Key Insights:")
        for insight in comparison['insights']:
            print(f"   • {insight}")
else:
    if not vbt_results['success']:
        print(f"⚠️ VectorBT not available: {vbt_results.get('error', 'Unknown error')}")
        print("Note: VectorBT requires installation in your environment")

In [ ]:
# Create side-by-side comparison visualization
print("📊 Creating VectorBT comparison plots...")

fig = create_vectorbt_comparison_plots(
    data=data,
    strategy=strategy,
    initial_capital=initial_capital,
    show_technical_indicators=True
)

if fig:
    print("✅ VectorBT comparison visualization created!")
    print("\n🔍 How to Interpret the Comparison:")
    print("\n   📈 Price Chart (Top Left):")
    print("      • Stock price with moving averages")
    print("      • Verify crossovers align with signals")
    
    print("\n   📊 Portfolio Performance (Top Right):")
    print("      • Blue = Our Framework")
    print("      • Red Dashed = VectorBT")
    print("      • Gray Dotted = Buy & Hold")
    print("      • Lines should be very close")
    
    print("\n   📉 Performance Metrics (Bottom Left):")
    print("      • Blue bars = Our Framework")
    print("      • Red bars = VectorBT")
    print("      • Should be similar heights")
    
    print("\n   📋 Speed & Status (Bottom Right):")
    print("      • Execution times comparison")
    print("      • Success status for both frameworks")
    
else:
    print("⚠️ Comparison visualization not available")
    print("This might happen if VectorBT is not installed")

## 6. Summary and Insights

In [ ]:
print("🎯 COMPLETE ANALYSIS SUMMARY")
print("=" * 50)

print(f"\n📊 Strategy Performance:")
print(f"   Symbol: {symbol}")
print(f"   Period: {data.index[0].strftime('%Y-%m-%d')} to {data.index[-1].strftime('%Y-%m-%d')}")
print(f"   Strategy: Moving Average ({strategy.params['short_window']}/{strategy.params['long_window']})")
print(f"   Total Return: {total_return:.2f}%")
print(f"   Buy & Hold: {buy_hold_return:.2f}%")
print(f"   Outperformance: {total_return - buy_hold_return:+.2f}%")

if 'comparison' in locals() and comparison['status']:
    print(f"\n🔄 Cross-Validation:")
    print(f"   Status: {comparison['status']}")
    print(f"   Framework Agreement: {comparison['correlation']:.3f}")
    if comparison['status'] == 'PASS':
        print("   ✅ Implementation validated successfully!")
    else:
        print("   ⚠️ Minor differences detected - review recommended")

print(f"\n💡 Key Learning Points:")
print(f"   ✅ Built complete backtesting framework from scratch")
print(f"   ✅ Tested with real market data (Yahoo Finance)")
print(f"   ✅ Cross-validated against industry standard (VectorBT)")
print(f"   ✅ Comprehensive visualization and analysis tools")
print(f"   ✅ Professional-grade performance metrics")

print(f"\n🚀 Next Steps:")
print(f"   • Try different strategy parameters")
print(f"   • Test other strategies (Momentum, Mean Reversion)")
print(f"   • Experiment with different assets")
print(f"   • Add risk management features")
print(f"   • Parameter optimization with VectorBT speed")

print(f"\n📚 Framework Benefits Demonstrated:")
print(f"   ✅ Beginner-friendly (no short selling)")
print(f"   ✅ Realistic execution costs")
print(f"   ✅ Professional visualization")
print(f"   ✅ Cross-validation capability")
print(f"   ✅ Easy to extend and customize")

## 🎉 Congratulations!

You've successfully:

1. **Built a Complete Trading System**: From data fetching to result visualization
2. **Implemented Professional Features**: Realistic costs, risk management, comprehensive metrics
3. **Validated Your Implementation**: Cross-checked against industry-standard VectorBT library
4. **Created Publication-Quality Visualizations**: Charts suitable for research and presentations

### 🔍 What the Cross-Validation Proves:
- Your strategy logic is implemented correctly
- Performance calculations are accurate
- Signal generation timing is precise
- Results can be trusted for decision-making

### 📈 Framework Advantages:
- **Custom Control**: Full control over execution logic
- **Educational Value**: Understand every component
- **Flexibility**: Easy to add new features
- **Validation**: Cross-check with professional tools

### ⚠️ Important Reminder:
This is for **educational purposes only**. Always:
- Test thoroughly before any real trading
- Start with paper trading
- Consult financial professionals
- Past performance ≠ future results